<a href="https://colab.research.google.com/github/December04/MachineLearningProgramming/blob/main/Week10/Assignment05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transformer Encorder

In [43]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models

In [45]:
HAR_PATH = "/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset/UCI HAR Dataset"

SIGNALS = [
    "total_acc_x","total_acc_y","total_acc_z",
    "body_acc_x","body_acc_y","body_acc_z",
    "body_gyro_x","body_gyro_y","body_gyro_z"
]

In [46]:
def load_signals(group):
    loaded = []
    for name in SIGNALS:
        filepath = os.path.join(HAR_PATH, group, "Inertial Signals", f"{name}_{group}.txt")
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"파일 없음: {filepath}")
        data = pd.read_csv(filepath, sep=r"\s+", header=None).values
        loaded.append(data)
    # (9, n_samples, 128) -> (n_samples, 128, 9)
    return np.transpose(np.array(loaded), (1, 2, 0))

In [47]:
def load_labels(group):
    filepath = os.path.join(HAR_PATH, group, f"y_{group}.txt")
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"레이블 파일 없음: {filepath}")
    labels = pd.read_csv(filepath, header=None)[0].values - 1
    return labels

In [48]:
X_train = load_signals("train")
X_test = load_signals("test")
y_train = load_labels("train")
y_test = load_labels("test")

print("데이터 로드 완료")
print("X_train:", X_train.shape)  # (7352,128,9)
print("X_test :", X_test.shape)   # (2947,128,9)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)

데이터 로드 완료
X_train: (7352, 128, 9)
X_test : (2947, 128, 9)
y_train: (7352,)
y_test : (2947,)


In [49]:
def transformer_encoder(inputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.1):
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(inputs + x)

    # Feed-forward
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.Dropout(dropout)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return x

In [50]:
def build_transformer_har_model(input_shape=(128,9), num_classes=6):
    inputs = layers.Input(shape=input_shape)

    # 간단한 Conv1D embedding
    x = layers.Conv1D(64, 3, padding="same", activation="relu")(inputs)

    # Transformer Encoder 2개
    x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)
    x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)

    # Pooling + Dense
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs, name="transformer_har_model")
    return model

In [51]:
num_classes = len(np.unique(y_train))
model = build_transformer_har_model((128,9), num_classes)
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0007),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

Model: "transformer_har_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 9)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 128, 64)   │      1,792 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 64)   │     66,368 │ conv1d_6[0][0],   │
│ (MultiHeadAttentio… │                   │            │ conv1d_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 128, 64)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 128, 64)   │          0 │ conv1d_6[0][0],   │
│                     │                   │            │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 128, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 128, 64)   │      8,256 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 128, 64)   │          0 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_6[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 64)   │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 128, 64)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_7[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 128, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 128, 64)   │      8,256 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 128, 64)   │          0 │ dense_15[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_11[0][0]

 Total params: 172,742 (674.77 KB)

 Trainable params: 172,742 (674.77 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    verbose=1
)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 93s 907ms/step - accuracy: 0.5922 - loss: 1.0696 - val_accuracy: 0.8334 - val_loss: 0.4339
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 137s 856ms/step - accuracy: 0.9070 - loss: 0.2646 - val_accuracy: 0.8858 - val_loss: 0.3648
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 85s 884ms/step - accuracy: 0.9485 - loss: 0.1415 - val_accuracy: 0.8865 - val_loss: 0.3792
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 80s 870ms/step - accuracy: 0.9484 - loss: 0.1318 - val_accuracy: 0.9082 - val_loss: 0.3983
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 80s 870ms/step - accuracy: 0.9522 - loss: 0.1025 - val_accuracy: 0.8831 - val_loss: 0.4647
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 79s 861ms/step - accuracy: 0.9580 - loss: 0.1064 - val_accuracy: 0.8939 - val_loss: 0.4621
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 87s 907ms/step - accuracy: 0.9600 - loss: 0.0960 - val_accuracy: 0.9021 - val_loss: 0.4720
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 80s 862ms/step - accuracy: 0.9511 - loss: 0.1092 - val_acc

In [53]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n Transformer HAR Test Accuracy: {acc:.4f}")


 Transformer HAR Test Accuracy: 0.8897
